In [ ]:
pip install boto3

In [ ]:
import boto3

In [ ]:
client = boto3.client('sagemaker')


In [ ]:
list_pipelines = client.list_pipelines().get('PipelineSummaries')
pipelines = [pipeline.get('PipelineName') for pipeline in list_pipelines]

In [ ]:
for pipeline in pipelines:
    client.delete_pipeline(PipelineName=pipeline_name)
    print("Deleted pipeline: ", pipeline)

In [ ]:
list_experiments = client.list_experiments().get('ExperimentSummaries')
experiments = [experiment.get('ExperimentName') for experiment in list_experiments]

In [ ]:
for experiment in experiments:
    list_trials = client.list_trials(ExperimentName=experiment)['TrialSummaries']
    trails = [trial.get('TrialName') for trail in list_trials]
    for trail in trails:
        list_trial_components = client.list_trail_components(TrailName=trail)['TrialComponentSummaries']
        for trail_component in list_trail_components:
            trial_component_name = trail_component.get('TrialComponentName')
            client.disassociate_trial_component(TrialComponentName=trial_component_name, TrialName=trail)
            try:
                client.delete_trial_component(TrialComponentName=trial_component_name)
                print("Deleted trail component: ", trial_component_name)
            except:
                # component is associated with another trial
                continue 
        client.delete_trail(TrialName=trail)
        print("Deleted trail: ", trail)
    client.delete_experiment(ExperimentName=experiment)
    print("Deleted experiment: ", experiment)